In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing as per
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [2]:
# Creare dataframe
tabel = pd.read_excel('FIFA.xlsx', index_col=None, na_values=['NA'])
print(tabel)
tabel.columns = [col.strip() for col in tabel.columns]
print(tabel[tabel['Nationality']== 'Romania'])
print(tabel.groupby(['Nationality']).count())
# Valori lipsa
print(tabel.isnull())
print(tabel.isnull().sum())

FileNotFoundError: [Errno 2] No such file or directory: 'FIFA.xlsx'

In [ ]:
print(tabel.drop(columns=['Special', 'Preferred Foot', 'International Reputation', 'Weak Foot', 'Work Rate', 'Body Type', 'Real Face', 'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until', 'Height', 'Weight', 'Release Clause' ]))
tabeldrop = pd.DataFrame(tabel.drop(columns=['Special', 'Preferred Foot', 'International Reputation', 'Weak Foot', 'Work Rate', 'Body Type', 'Real Face', 'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until', 'Height', 'Weight', 'Release Clause' ]))

In [ ]:
# Selectam jucatorii cu 'Overall rating' mai mare de 66
tabeldrop= tabeldrop[tabeldrop['Overall rating'] >66]
print(tabeldrop)

In [ ]:
# Valori lipsa (vedem unde avem valori lipsa)
print(tabeldrop.isnull())
print(tabeldrop.isnull().sum())

In [ ]:
# Inlocuim valorile lipsa ale variabilei 'Club' cu 'No club'
print(tabeldrop['Club'].fillna(value='No club', inplace=True))
print(tabeldrop['Club'].value_counts(dropna=False))

In [ ]:
# Intrucat singurele campuri ramase fara valori sunt cele 3 de la variabila 'Position', eliminam randurile unde exista cel putin o variabila lipsa pentru a scapa de ele
print(tabeldrop.dropna(how='any', axis=0))
tabelfinal= pd.DataFrame(tabeldrop.dropna(how='any', axis=0))
print(tabelfinal)
print(tabelfinal['Club'].value_counts(dropna=False))
print(tabelfinal)
# tabelfinal este dataframe-ul curatat si aranjat cu care vom lucra mai departe

In [ ]:
# Cream un dataframe cu variabilele cantitative pe care vrem sa le includem in ACP
print(tabelfinal.drop(columns=['ID', 'Name', 'Age', 'Nationality', 'Overall rating', 'Club', 'Value (M euros)', 'Wage (K euros)', 'Skill Moves', 'Position']))
tabelACP= pd.DataFrame(tabelfinal.drop(columns=['ID','Name', 'Age', 'Nationality', 'Overall rating', 'Club', 'Value (M euros)', 'Wage (K euros)', 'Skill Moves', 'Position', 'Curve', 'SprintSpeed', 'Balance', 'Jumping', 'Aggression', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Crossing', 'Volleys', 'Stamina','HeadingAccuracy', 'Strength', 'Interceptions', 'LongShots']))
print(tabelACP)

In [ ]:
# Matrice de corelatie (o folosim sa vedem daca ACP ar da rezultate bune)
corr = tabelACP.corr()
print(corr)
fig = plt.figure(figsize=(10,10))
plt.matshow(corr, cmap = 'RdBu', fignum = fig.number)
plt.xticks(range(len(corr.columns)), corr.columns, rotation = 'vertical')
plt.yticks(range(len(corr.columns)), corr.columns)
plt.show()

In [ ]:
tabelACP1= pd.DataFrame(tabelfinal.drop(columns=['ID', 'Name', 'Nationality', 'Club', 'Value (M euros)',  'Position', 'Curve', 'SprintSpeed', 'Balance', 'Jumping', 'Aggression', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Crossing', 'Volleys', 'Stamina','HeadingAccuracy', 'Strength', 'Interceptions', 'LongShots']))
print(tabelACP1)

In [ ]:
# Standardizare date
scaler = per.MinMaxScaler(feature_range= (0,1))
rescaleData = scaler.fit_transform(tabelACP1)
rescaleData = pd.DataFrame(rescaleData, index = tabelACP1.index, columns = tabelACP1.columns)
print(rescaleData)

In [ ]:
# ACP
pca= PCA(n_components=2)
print(pca.fit(rescaleData))
pcaData= pca.transform(rescaleData)
print(pcaData)

In [ ]:
# In dataframe-ul pcaData putem observa valorile Eigenvectorilor si faptul ca PC2 e determinata pozitiv de jucatorii care au trasaturi ofensive
# PC1 este determinata pozitiv de jucatorii care au trasaturi defensive
pcaData= pd.DataFrame(pca.transform(rescaleData))
print(pcaData.rename(columns= {0:'PC1', 1:'PC2'}, inplace=True))
print(pcaData)
print(pcaData.merge(right= tabelfinal['Name'], left_index=True, right_index=True))
tabelplot= pd.DataFrame(pcaData.merge(right= tabelfinal['Name'], left_index=True, right_index=True))
print(tabelplot.set_index('Name'))
df_eigenvectots= pd.DataFrame(tabelplot.set_index('Name'))
df_eigenvectots.to_csv('Eigenvectors.csv')

In [ ]:
# Cream un Screeplot
per_var= np.round(pca.explained_variance_ratio_*100, decimals=1)
labels= ['PC' + str(x) for x in range(1, len(per_var)+1)]
plt.bar(x= range(1,len(per_var) + 1), height= per_var, tick_label= labels)
plt.ylabel('Percentage of Explained Variance')
plt.xlabel('Principal Component')
plt.title('Scree Plot')
plt.show()

In [ ]:
# Cream un PCA plot

tabelplot.plot(kind = 'scatter', x = 'PC1', y = 'PC2')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('PCA Plot')
plt.show()

In [ ]:
print(rescaleData)
print(rescaleData.drop([8760, 8761, 8762], axis=0))
rescaleData1= pd.DataFrame(rescaleData.drop([8760, 8761, 8762], axis=0))
print(tabelplot)

In [ ]:
print(rescaleData1.merge(right= tabelplot, left_index=True, right_index=True))
df_last= pd.DataFrame(rescaleData1.merge(right= tabelplot, left_index=True, right_index=True))
df_last.set_index('Name', inplace=True)
print(df_last.columns)

In [ ]:
# Clusterizare
cluster= KMeans(n_clusters=5)
df_last['cluster']= cluster.fit_predict(df_last[df_last.columns[:]])
print(df_last['cluster'].value_counts())
print(df_last[df_last['cluster']==0])

In [ ]:
# Distributia salariilor saptamanale
tabelfinal['Wage (K euros)'].plot(kind = 'hist', edgecolor = 'k', figsize = (14,6))
plt.xlabel('Weekly wage')
plt.ylabel('Frequency')
plt.title('Distribution of Wages')
plt.show()